# Connect to Spotify API via spotipy wrapper

In [1]:
from getpass import getpass
from tqdm.notebook import tqdm
from tqdm.auto import tqdm

In [2]:
c_id = getpass("client id?")
c_se = getpass("client secret?")

client id?········
client secret?········


In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

#Initialize SpotiPy with user credentials
# sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=c_id,
#                                                           client_secret=c_se))

Goal: a dataframe:

In [4]:
### Get song id's from playlist

SyntaxError: invalid syntax (<ipython-input-4-79b3e162b6a9>, line 1)

In [31]:
# make it work for one
playlist = sp.user_playlist_tracks("spotify", "4rnleEAOdmFAbRcNCgZMpY")

In [32]:
tracklist = []
for track in playlist['items']:
    tracklist.append(track['track']['id'])
len(tracklist)

100

In [162]:
# import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import requests


class SongCollector:
    def __init__(self, dataframe=pd.DataFrame()):
        self.df = dataframe
        self.playlist_ids = []
        self.categories = []
        self.category_ids = []
        self.c_id = ""
        self.c_se = ""
        self.sp = spotipy.Spotify()
        self.tracks = []
        self.track_ids = []
        self.album_uris = []
        self.track_ids_for_all_albums_found = []
        
    def setup_credentials(self, c_id, c_se):
        # self.c_id = input("Spotify client ID?")
        # self.c_se = input("Spotify client secret?")
        self.c_id = c_id
        self.c_se = c_se
        
        self.sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=c_id, client_secret=c_se))
        
    def load_playlist_ids(self, playlist_ids):
        self.playlists = playlist_ids
        

#     def collect_songs_from_playlists(self,
#                                      remove_duplicates=False):
#         """
#         Takes the dataframe stored in self.df and expands it by the songs 
#         provided by the user in the list <playlists>
        
#         playlists: A list of spotify playlist URIs as strings
#         """
#         for playlist_id in tqdm(self.playlist_ids):
#             self.get_playlist_tracks(playlist_id)
#             try:
#                 playlist_df = self.create_df_from_tracks(self.tracks)
#                 self.df = pd.concat([self.df, playlist_df], axis=0)
#             except (requests.HTTPError, spotipy.client.SpotifyException) as exception:
#                 # print(exception)
#                 continue
            
#         # the robot should return the resulting dataframe
#         return self.df
    
    def perform_full_take(self, deep_lookup=False, to_pkl=False):
        """Performs a "full take" of as many songs that can be found via the spotify API.
        
        Starting from Spotify's categories, all category ids are collected. For those, all associated
        playlist ids are collected and ultimatey in each of those playlists all unique track ids are collected.
        
        If the argument `deep_lookup` is set to True, in a next step for every found track id the associated
        album id is searched and the track ids of those are also added.

        Parameters
        ----------
        deep_lookup : bool, optional
            toggles whether deep lookup should be performed
            
        to_pkl : bool, optional
            toggles whether track ids should be saved to .pkl file in ./data

        """
        
        # check for sane credentials
        self.check_authmanager()
        
        # get all available category id's
        self.category_ids = [cat['id'] for cat in self.sp.categories()['categories']['items']]
        
        # now get & save all playlist id's from those categories
        for cat_id in self.category_ids:
            # catch HTTP error for some categories
            try:
                pl_per_category_json = self.sp.category_playlists(cat_id)['playlists']['items']
            except spotipy.client.SpotifyException as e:
                continue
                
            for pl in pl_per_category_json:
                try:
                    id_ = pl['id']
                except TypeError:
                    continue
                self.playlist_ids.append(f"spotify:playlist:{id_}")
        
        # populating the track_id's with the tracks found in the playlist_ids
        for playlist_id in self.playlist_ids:
            self.track_ids.extend(self.get_track_ids_from_playlist_id(playlist_id))
        
        self.track_ids = list(set(self.track_ids))
        print(f"{len(self.track_ids)} unique track ids found")
        
        
        # if deep_lookup
        if deep_lookup:
            print("starting deep lookup...")
            print("now getting all album ids for every song")
            for track_id in tqdm(self.track_ids):
                self.album_uris.append(sp.track(track_id)['album']['uri'])

            self.album_uris = list(set(self.album_uris))
            print(f"{len(self.album_uris)} unique album uris found")
            print("----")
            print("Getting all the songs from those uris")

            self.track_ids_for_all_albums_found = list(set(get_track_ids_from_album_uris(self.album_uris)))

            print(f"After deep lookup: {len(self.track_ids_for_all_albums_found)} unique track ids found")
        

    def get_track_ids_from_album_uris(self, album_uris):
        """Returns a list of track ids for every album uri provided as an input list

        Parameters
        ----------
        album_uris : list
            A list of Spotify album URIs
            
        to_pkl : bool, optional
            toggles whether track ids should be saved to .pkl file in ./data
            
        Returns
        ----------
        track_ids : list
            A list of spotify track ids

        """
        
        track_ids = []
        for album_uri in tqdm(album_uris, disable=False):
            tracks = self.sp.album_tracks(album_uri)
            for track in tracks['items']:
                track_ids.append(track['id'])
        return track_ids
        
#### FIX FROM HERE
        
    
    def get_track_ids_from_playlist_id(self, playlist_id):
        """
        Returns the track ids from one playlist_id
        """
        
        results = self.sp.user_playlist_tracks("spotify", playlist_id)
        tracks = results['items']
        while results['next']:
            results = self.sp.next(results)
            tracks.extend(results['items'])
            
        # keep only those tracks with ids
        tracks = [track for track in tracks if self.track_exists(track) and self.id_exists(track)]
        
        return [track['track']['id'] for track in tracks]

#     def create_df_from_tracks(self, tracks):
#         """
#         returns an audio feature dataframe for the input tracks
#         """
        
        
#         # making sure we only get track who really have an id
#         tracks = [track for track in tracks if self.track_exists(track) and self.id_exists(track)]
#         #tracks = [track for track in tracks if id_exists(track)]

#         # getting all the ids from the tracks
#         ids = [self.get_id(track) for track in tracks]

#         # Getting the song features from the song ids
#         # Warning: Only 100 tracks here

#         features = self.sp.audio_features(ids)

#         try:
#             df = pd.DataFrame({'name': [track['track']['name'] for track in tracks],
#                                # a track can have mulitple artists. if so, add them seperated by ","
#                                'artists': [", ".join([artist['name'] for artist in track['track']['artists']]) for track in tracks],
#                                # 'ids': ids,
#                               })

#             # Creating a dataframe from the features 
#             df_features = pd.DataFrame(features)

#             return pd.concat([df, df_features], axis=1)

#         except AttributeError:
#             return pd.DataFrame(columns=['name',
#                                          'artists',
#                                          'danceability',
#                                          'energy',
#                                          'key',
#                                          'loudness',
#                                          'mode',
#                                          'speechiness',  
#                                          'acousticness',
#                                          'instrumentalness',
#                                          'liveness',
#                                          'valence',
#                                          'tempo',
#                                          'type',
#                                          'id',
#                                          'uri',
#                                          'track_href',
#                                          'analysis_url',
#                                          'duration_ms',
#                                          'time_signature'],
#                                )

    def get_id(self, track):
            return track['track']['id']

    def id_exists(self, track):
        return bool(get_id(track))

    def track_exists(self, track):
        return track['track']


    def check_authmanager(self):
        """
        TODO: include this into other functions
        """
        if not self.sp.auth_manager:
            print("spotipy.Spotify().auth_manager not set up. Run setup_credentials()?")

In [163]:
# playlists = ["6VUYYbBUbRDyWMszBZZ8yP"]

In [164]:
my_songcollector = SongCollector()
# my_songcollector.collect_songs_from_playlists(playlists)

In [165]:
my_songcollector.setup_credentials("64c2853e1fdd437f881a7e4623735b4f",
                                  "4de3828cde74459d9765d1dce090e8d7")

#my_songcollector.get_playlist_tracks("6VUYYbBUbRDyWMszBZZ8yP")[0]['track']['id']

In [167]:
my_songcollector.perform_full_take(deep_lookup=True)

17863 unique track ids found
starting deep lookup...
now getting all album ids for every song


KeyboardInterrupt: 

In [157]:
len(my_songcollector.album_uris)

13636

In [158]:
track_ids = []
for album_uri in tqdm(my_songcollector.album_uris):
    tracks = sp.album_tracks(album_uri)
    for track in tracks['items']:
        track_ids.append(track['id'])
print(len(track_ids))


101410


In [ ]:
import pandas as pd

In [168]:
tracks_df = pd.DataFrame({'track_ids':track_ids})

In [171]:
tracks_df.to_pickle("data/track_ids.pkl")

In [148]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials("64c2853e1fdd437f881a7e4623735b4f",
                                  "4de3828cde74459d9765d1dce090e8d7"))



# album_uri = sp.track('7BoobGhD4x5K96Me0hqC8Q')['album']['uri']

# tracks = sp.album_tracks(album_uri)
# for track in tracks['items']:
#    print(track['name'])

Build a Bitch


In [127]:
# sp.category_playlists("chill")['playlists']['items']

In [121]:
# sp.category_playlists("party")['playlists']['items']

In [115]:
my_song_collector

HTTP Error for GET to https://api.spotify.com/v1/browse/categories/the_local_hub/playlists returned 404 due to Specified id doesn't exist


SpotifyException: http status: 404, code:-1 - https://api.spotify.com/v1/browse/categories/the_local_hub/playlists?limit=20&offset=0:
 Specified id doesn't exist, reason: None

In [151]:
[cat['id'] for cat in sp.categories()['categories']['items']]

['toplists',
 'summer',
 'pop',
 'mood',
 'hiphop',
 'workout',
 'edm_dance',
 'party',
 'alternative',
 'at_home',
 'wellness',
 'rock',
 'frequency',
 'indie_alt',
 'pride',
 'equal',
 'rnb',
 'throwback',
 'chill',
 'kids_family']

In [157]:
sp.categories()['categories']['items'][17]

{'href': 'https://api.spotify.com/v1/browse/categories/throwback',
 'icons': [{'height': None,
   'url': 'https://t.scdn.co/images/448949e524284cea9aedbf32aa69d90f.jpeg',
   'width': None}],
 'id': 'throwback',
 'name': 'Throwback'}

In [178]:
julia_playlist_id = "5S8SJdl1BDc0ugpkEvFsIL"

In [184]:
results = sp.user_playlist_tracks('spotify', julia_playlist_id)
tracks = results['items']
while results['next']:
    results = sp.next(results)
    tracks =

In [187]:
sp.next(results)['next']

'https://api.spotify.com/v1/playlists/5S8SJdl1BDc0ugpkEvFsIL/tracks?offset=200&limit=100&additional_types=track'

In [189]:
while "":
    print("bla")

In [190]:
bool("")

False

In [191]:
SongCollector.df

AttributeError: type object 'SongCollector' has no attribute 'df'

In [235]:
def get_all_tracks(album_ids):
    all_tracks = []
    for album_id in album_ids:
        track_list = sp.album_tracks(album_id, limit=50)
        #print(track_list)
        all_tracks_album = track_list['items']
        
        while track_list['next']:
            track_list = sp.next(track_list)
            all_tracks_album.extend(track_list['items'])
        all_tracks.extend(all_tracks_album)
    return all_tracks
        

In [236]:
al_ids = [
         "3osvHnt9ncEEOAGuCIESaZ",
         "4YSmNOiY47nx2VodZ8N1I8"]

In [237]:
all_tracks = get_all_tracks(al_ids)

In [173]:
a = 5

print("After deep lookup: "
    f"{a} unique"
    " track ids found"
    )

After deep lookup: 5 unique track ids found


In [208]:
sp.album_tracks("3osvHnt9ncEEOAGuCIESaZ", limit=50)

{'href': 'https://api.spotify.com/v1/albums/3osvHnt9ncEEOAGuCIESaZ/tracks?offset=0&limit=50',
 'items': [{'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3Cv5jiNHUHUvWwdmclhgOe'},
     'href': 'https://api.spotify.com/v1/artists/3Cv5jiNHUHUvWwdmclhgOe',
     'id': '3Cv5jiNHUHUvWwdmclhgOe',
     'name': 'Holographic Field',
     'type': 'artist',
     'uri': 'spotify:artist:3Cv5jiNHUHUvWwdmclhgOe'}],
   'available_markets': ['AD',
    'AE',
    'AG',
    'AL',
    'AM',
    'AO',
    'AR',
    'AT',
    'AU',
    'AZ',
    'BA',
    'BB',
    'BD',
    'BE',
    'BF',
    'BG',
    'BH',
    'BI',
    'BJ',
    'BN',
    'BO',
    'BR',
    'BS',
    'BT',
    'BW',
    'BY',
    'BZ',
    'CA',
    'CH',
    'CI',
    'CL',
    'CM',
    'CO',
    'CR',
    'CV',
    'CW',
    'CY',
    'CZ',
    'DE',
    'DJ',
    'DK',
    'DM',
    'DO',
    'DZ',
    'EC',
    'EE',
    'EG',
    'ES',
    'FI',
    'FJ',
    'FM',
    'FR',
    'GA',
    'GB',
    'GD',

In [161]:

for i in tqdm(range(10), disable=True):
    print(i)

0
1
2
3
4
5
6
7
8
9


In [ ]:
from tqdm.auto import tqdm